In [27]:
def arithmetic_tool(operation: str, a: float, b: float) -> float:
    """
    Perform basic arithmetic operations.
    
    Args:
        operation (str): One of '+', '-', '*', '/', '**', '%'
        a (float): First operand
        b (float): Second operand

    Returns:
        float: Result of the operation

    Raises:
        ValueError: If operation is unsupported or division by zero
    """
    operations = {
        '+': lambda x, y: x + y,
        '-': lambda x, y: x - y,
        '*': lambda x, y: x * y,
        '/': lambda x, y: x / y if y != 0 else (_ for _ in ()).throw(ValueError("Division by zero")),
        '**': lambda x, y: x ** y,
        '%': lambda x, y: x % y if y != 0 else (_ for _ in ()).throw(ValueError("Modulo by zero")),
    }

    if operation not in operations:
        raise ValueError(f"Unsupported operation: {operation}")

    return operations[operation](a, b)

In [28]:
def get_weather(place: str) -> int:
    """Return the current temperature in Fahrenheit for a given place."""
    place = place.lower().strip()  # Normalize input
    if place == "newyork":
        return 55  # ~13°C — typical spring/fall day
    elif place == "chicago":
        return 30  # ~ -1°C — plausible winter day
    else:
        return 40  # Default fallback (~4°C)        

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

In [31]:
model1 = ChatOpenAI(model="gpt-4o")
model2 = ChatOpenAI(model="gpt-4o")
model3 = ChatOpenAI(model="gpt-4o")

In [32]:
# welcome_agent
# calculator_agent
# weather_agent

In [33]:
welcome_agent_tools = [create_handoff_tool(agent_name="calculator_agent"), create_handoff_tool(agent_name="weather_agent")]

In [34]:
welcome_agent = create_react_agent(
    model=model1.bind_tools(welcome_agent_tools, parallel_tool_calls=False), 
    tools=welcome_agent_tools, 
    prompt="You are a welcome agent, Whenever a greetings says, you need to do the greeting back", 
    name="welcome_agent"
)

In [35]:
calculator_agent_tools = [arithmetic_tool, create_handoff_tool(agent_name="welcome_agent"), create_handoff_tool(agent_name="weather_agent")]
calculator_agent = create_react_agent(
    model=model2.bind_tools(calculator_agent_tools, parallel_tool_calls=False), 
    tools=calculator_agent_tools,
    prompt="Calculator agent for add, subtract, multiplication and division", 
    name="calculator_agent"
)

In [36]:
weather_agent_tool =[get_weather, create_handoff_tool(agent_name='welcome_agent'), create_handoff_tool(agent_name="calculator_agent")]
weather_agent = create_react_agent(
    model=model3.bind_tools(weather_agent_tool, parallel_tool_calls=False), 
    tools=weather_agent_tool, 
    prompt="YOu are responsible to return weather in a place", 
    name="weather_agent"
)

In [37]:
workflow = create_swarm(
    [calculator_agent, welcome_agent, weather_agent],
    default_active_agent="welcome_agent"
)

In [38]:
from langgraph.checkpoint.memory import InMemorySaver

In [39]:
checkpointer = InMemorySaver()

In [40]:
langgraph_app = workflow.compile(checkpointer=checkpointer)

In [51]:
config = {"configurable": {"thread_id": "10"}}

In [56]:
turn = langgraph_app.invoke({
    "messages": [{"role": "user", "content": "Add 30 and 30"}]
}, config)

In [57]:
turn

{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='f07df76c-1886-4b64-a2e4-35df6c4e71cb'),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 82, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-CIA2RxWUtFAm3JxKmHJgRxKMcE8AE', 'finish_reason': 'stop', 'logprobs': None}, name='welcome_agent', id='run--76123622-2653-43e8-bc2b-5f2e750f491a-0', usage_metadata={'input_tokens': 82, 'output_tokens': 10, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='Give me weather in c

In [59]:
import asyncio

async for event in langgraph_app.astream_events(
    {
        "messages": [
            {"role": "user", "content": "Give me sum of Weather in newyork and chicago"}
        ]
    },
    config,
):
    if event.get("event") == "on_chat_model_stream":
        data = event.get("data", {})
        chunk = data.get("chunk")

        if not chunk:
            continue

        content = getattr(chunk, "content", "")
        additional_kwargs = getattr(chunk, "additional_kwargs", {})
        response_metadata = getattr(chunk, "response_metadata", {})

        if content and additional_kwargs == {}:
            print(content, end="", flush=True)
            await asyncio.sleep(0.01)  # ✅ async-friendly sleep

        if response_metadata.get("finish_reason") == "stop":
            print()
            break


The sum of the temperatures in New York (40°F) and Chicago (30°F) is 70°F.
